In [1]:
# import necessary libraries
import pandas as pd
import os
import textstat
from openai import OpenAI
import json
import re
import requests
from dotenv import load_dotenv

In [25]:
# import prompts 
from jh_pfx_prompts import example, icd10_example, baseline_zeroshot_prompt, single_fewshot_icd10_labeling_prompt

In [40]:
os.environ['OPENAI_API_KEY'] = ''

In [4]:
# api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLIENT = OpenAI(api_key = OPENAI_API_KEY)
OPENAI_MODEL = "gpt-4o"

In [5]:
#reading levels
PROFESSIONAL = "Professional"
COLLEGE_GRADUATE = "College Graduate"
COLLEGE = "College"
TENTH_TO_TWELTH_GRADE = "10th to 12th grade"
EIGTH_TO_NINTH_GRADE = "8th to 9th grade"
SEVENTH_GRADE = "7th grade"
SIXTH_GRADE = "6th grade"
FIFTH_GRADE = "5th grade"
N_A = "N/A"

In [6]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return PROFESSIONAL
    elif 10.0 <= flesch_reading_ease < 30.0:
        return COLLEGE_GRADUATE
    elif 30.0 <= flesch_reading_ease < 50.0:
        return COLLEGE
    elif 50.0 <= flesch_reading_ease < 60.0:
        return TENTH_TO_TWELTH_GRADE
    elif 60.0 <= flesch_reading_ease < 70.0:
        return EIGTH_TO_NINTH_GRADE
    elif 70.0 <= flesch_reading_ease < 80.0:
        return SEVENTH_GRADE
    elif 80.0 <= flesch_reading_ease < 90.0:
        return SIXTH_GRADE
    elif 90.0 <= flesch_reading_ease < 100.0:
        return FIFTH_GRADE 
    else:
        return N_A

In [7]:
# import fewshot examples
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [8]:
# import evaluation data 
df_eval = pd.read_csv('pfx_evaluation_data.csv')

In [9]:
# extract the json from openai
def extract_json(openai_response):
    if openai_response:  # Ensure the response is not None
        try:
            # Extract content from response object
            content = openai_response.message.content
            
            # Search for JSON within the content
            json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)  # Parse JSON string to Python dict
            else:
                print("No JSON found in response content.")
                return None
        except AttributeError as e:
            print(f"Attribute error: {e}. Ensure the input is a valid response object.")
            return None
    else:
        return None

In [30]:
# takes json object and generated icd10 labels for incidental findings 
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    pfx_icd10_codes = []
    pfx_icd10_response = None 
    for pfx_output in pfx_outputs_json :
        try:
            prompt = single_fewshot_icd10_labeling_prompt.format(examples = pfx_icd10_fewshot_examples, PFx = pfx_output['PFx'])
        except:
            print("ERROR: %S" % pfx_output)
            return
        pfx_icd10_response = CLIENT.chat.completions.create(
            model = OPENAI_MODEL,
            temperature = 0.0,
            messages = [
                {"role": "system", "content": "You are an ICD10 medical coder for incidental findings. "},
                {"role": "system", "content": prompt}
            ],
            stream = False,
        )
    pfx_icd10_codes.append(pfx_icd10_response.choices[0])
    return list(map(extract_json, pfx_icd10_codes))

In [11]:
# gets zeroshot response 
pfx_zeroshot_outputs = []
for i, row in df_eval.iloc[:1].iterrows():
    prompt = baseline_zeroshot_prompt.format(Incidental_Finding = row['Incidental_Finding'], Reading_Level = TENTH_TO_TWELTH_GRADE)
pfx_response = None 
pfx_response = CLIENT.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = 0.0,
    messages = [
        {"role": "system", "content": "You are a md doctor rephrasing and explaining medical terminology to a patient in an understandable manor."},
        {"role": "system", "content": prompt}
    ],
    stream = False,
)

pfx_zeroshot_outputs.append(pfx_response.choices[0])

In [12]:

# converts to a readable format 
pfx_zeroshot_outputs_json = list(map(extract_json, pfx_zeroshot_outputs))

In [15]:
pfx_zeroshot_outputs_json

[{'finding': 'White matter lesions',
  'ICD10_code': 'R90.89',
  'PFx': "White matter lesions are areas in the brain where the tissue has changed, often due to aging, high blood pressure, or other health conditions. These changes can sometimes affect how well the brain works, but they don't always cause symptoms. It's like having a few spots on a photograph; they might not change the overall picture, but they are noticeable. It's important to keep an eye on your overall health, manage any conditions like high blood pressure, and maintain a healthy lifestyle to support brain health.",
  'PFx_ICD10_code': 'Z71.89'}]

In [28]:
pfx_zeroshot_output_df = pd.DataFrame(pfx_zeroshot_outputs_json)

In [31]:
pfx_zeroshot_outputs_icd10_labels = label_icd10s(pfx_zeroshot_outputs_json)

In [37]:
pfx_zeroshot_output_df['_0_agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "", pfx_zeroshot_outputs_icd10_labels))
pfx_zeroshot_output_df["_0_icd10_matches"]= pfx_zeroshot_output_df.ICD10_code == pfx_zeroshot_output_df._0_agent_icd10_codes
pfx_zeroshot_output_df["_0_flesch"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_zeroshot_output_df["_0_reading_level"] = pfx_zeroshot_output_df['_0_flesch'].apply(map_reading_level)
pfx_zeroshot_output_df["_0_flesch_kincaid"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_kincaid_grade)
pfx_zeroshot_output_df["_0_reading_level_matches"] = pfx_zeroshot_output_df['_0_reading_level'] == TENTH_TO_TWELTH_GRADE

In [38]:
# Calculate accuracy score
accuracy = sum(pfx_zeroshot_output_df["_0_icd10_matches"]) / len(pfx_zeroshot_output_df.index)
# Calculate readability score 
readability = pfx_zeroshot_output_df["_0_flesch_kincaid"]
# Compute the overall score
overall_score = (readability * 0.3) + (accuracy * 0.7)

grades_data = []
grades_data.append({
    "accuracy": float(accuracy),  # Assuming accuracy is already a scalar
    "readability": float(readability.iloc[0]),  # Use .iloc[0] to extract scalar
    "overall_score": float(overall_score.iloc[0])  # Use .iloc[0] to extract scalar
})
grades = pd.DataFrame(grades_data)


In [39]:
grades

,accuracy,readability,overall_score
0,1.0,9.3,3.49
